<a href="https://colab.research.google.com/github/Lucy-Moctezuma/SFSU-CodeLab-Work-/blob/main/E.%20Coli%20Machine%20Learning%20Project/3_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Random Forest**
## ***Objectives for this Notebook***
- Familiarize with the basics of how tree models works, specifically Random Forests.
- Implementing functions to run Random Forest to Moragadivand's dataset.

<a name="tree-methods"></a>
Random Forests belong to a group of algorithms known as tree methods. A Random Forest model is also an Ensemble method. These methods use an arrangement of weaker predictor models (in this case Decision Trees) to produce a stronger model (in this case Random Forest). The basis of all tree methods is the Decision Tree. For those unfamiliar with tree methods, we have some definitions below:

![tree_forest.png](https://drive.google.com/uc?export=view&id=1TBxIoL7K4WeXhegZZAvURJSxREw-Uqto)

- A **Decision Tree (left)** is a supervised learning algorithm that can be used for regression (predicting a continuous variable) or for classification (predicting a categorical variable). Our task is to classify each *E. coli* sample into Resistant (**R**) or Susceptible (**S**) classes. Decision Trees assign values to each feature input and compares them to a calculated threshold. We can picture a tree upside down, starting with the root on top and stopping at the leafs at the bottom, the different squares represent decision thresholds, if the isolate passes the threshold, then we move down the tree following the "yes" arrow, if not then we follow the "no" arrow to the next square down below until a classification decision is finally made.

- A **Random Forest (right)** is the combination of several Decision Trees outputs, each tree classifies all samples independently and the final decision for each sample is made based on the decision of the majority of trees. Each tree inside the Random Forest is created by selecting a random subset of features instead of using all the features and then it uses **BAGGING**.

  - **Bagging:** is a process of randomly selecting a subset of unique observations from the training data and then duplicating some of these selections until the number of observations used in one tree is the same as the number of all the original observations in the training data. Essentially it samples from original data with replacement and then agregates results for final prediction. Bagging tries to reduce overfitting and variance.
  
If we chose to use a combination of features such as: **GS** (Gene Presence or Absence + Population Structure), each tree would randomly use only some columns from the total amount of column features (G + S dataset).

We will explain further details on the structure of Random Forests as we code along to further clarify how nodes are chosen and how it uses the outputs of all trees for a final decision.








### **1) Importing Packages needed**

In [ ]:
# Data manipulation imports for ML
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Import packages for Random Forest model
from sklearn.ensemble  import RandomForestClassifier
from sklearn import tree

# Imports for model evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Imports for data visualization
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import graphviz

# Imports for file management
import os
from google.colab import drive
drive.mount('/content/drive')

### **2) Loading CSV file and creating dataframes for each antibiotic**

Similar to the previous notebook we loaded the dataframe created in the first notebook of this tutorial, then we create a dataframe for each antibiotic. To check that our function works we will test it using "CTX" antibiotic.

In [ ]:
# Loads csv file as a dataframe
filepath = '/content/drive/My Drive/EColi_ML_CSV_files/'

# reads csv file as a dataframe
All_Drugs_df = pd.read_csv(filepath+"EColi_Merged_dfs.csv", na_values="NaN")
All_Drugs_df.head()

### **3) Separating each Drug Dataframe into 4 sections : Training (features and labels) and Testing (features and labels)**

Similarly to the previous notebook, we create a dataframe and split it into training and testing chunks, each chunk with features and labels. Then we store these 4 sections into a a dictionary. We will then implement this function on the dataframe created in part 2.

In [ ]:
# Separating each dataframe into Labels and Features for training and testing data.
# Our function uses the handy train_test_split() function.

def Split_train_test(drug):
  #here we make a list of the columns we want to keep: the column for the isolate, the column for the drug we are interested in and all features (starting from column 13).
  df_list = [All_Drugs_df[["Isolate",drug]], All_Drugs_df.iloc[:,13:]]

  #here we create a data frame with just the columns we wanted to keep.
  Drug_df = pd.concat(df_list, axis=1)

  #here we drop all rows with missing data
  Drug_df = Drug_df.dropna()

  # Creating a dictionary to store each antibiotic datasets
  Train_test_dic = {}

  # Defining the label columns
  labels = Drug_df[drug]

  # Defining features columns
  features = Drug_df.drop(columns=[drug])

  # Separating training (features and labels) and testing (features and labels) datasets
  features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42, stratify=labels)

  # storing each data chunk in a dictionary
  Train_test_dic['labels_train'] = labels_train
  Train_test_dic['features_train'] = features_train
  Train_test_dic['labels_test'] = labels_test
  Train_test_dic['features_test'] = features_test

  return Train_test_dic

Below we implement the function **Split_train_test()** and then print how many are resistant and how many are Susceptible for the antibiotic CTX.

In [ ]:
# Implementing the function Split_train_test() for CTZ example
CTX_Train_test_dic = Split_train_test("CTX")

# checking the shape of each dataframe or series stored in the dictionary created for drug CTZ
print("CTX")
for k, df in CTX_Train_test_dic.items():
  print(k, df.shape)
  # counting how many of the labels have susceptible versus resistant ones
  if k.startswith("label"):
    print(df.value_counts())

Notice that just like before we have more *E. coli* samples that are Susceptible, rather than Resistant, therefore we need to remember to pay attention to the metrics of precision and recall for each of these classes rather than just looking at the overall accuracy. **The total number in our training observations is 1245.**

### **4) Creating different combination of features before training**

**NOTE:** Same code as in previous notebook

In [ ]:
# making a list of combinations of data sources we would like to test in our ML models
combo_list = ['G', 'S', 'GY', 'SY', 'GS', 'GYS']

# making a function that creates different feature combinations of the predictor features
def combo_feat(features_df, drug, combo):

  # creating Year column filters for features_df
  year_filter = [col for col in features_df if col.startswith("Year")]
  year_feat = features_df[year_filter]

  # creating Population structure column filters for features_df
  pop_str_filter = [col for col in features_df if col.startswith("cutoff")]
  pop_struc_feat = features_df[pop_str_filter]

  # creating Gene precence column filters for features_df
  gene_presc_filter = [col for col in features_df.columns if col not in pop_str_filter and col not in year_filter and col != "Isolate"]
  gene_presc_feat = features_df[gene_presc_filter]

  if combo == 'G':
    df_list = [features_df['Isolate'], gene_presc_feat]
    G_feat_df = pd.concat(df_list, axis=1)
    G_feat_df = G_feat_df.drop(columns=['Isolate'])
    return G_feat_df

  if combo == 'S':
    df_list = [features_df['Isolate'], pop_struc_feat]
    S_feat_df = pd.concat(df_list, axis=1)
    S_feat_df = S_feat_df.drop(columns=['Isolate'])
    return S_feat_df

  if combo == 'GY':
    df_list = [features_df['Isolate'], gene_presc_feat, year_feat]
    GY_feat_df = pd.concat(df_list, axis=1)
    GY_feat_df = GY_feat_df.drop(columns=['Isolate'])
    return GY_feat_df

  if combo == 'SY':
    df_list = [features_df['Isolate'], pop_struc_feat, year_feat]
    SY_feat_df = pd.concat(df_list, axis=1)
    SY_feat_df = SY_feat_df.drop(columns=['Isolate'])
    return SY_feat_df

  if combo == 'GS':
    df_list = [features_df['Isolate'], gene_presc_feat, pop_struc_feat]
    GS_feat_df = pd.concat(df_list, axis=1)
    GS_feat_df = GS_feat_df.drop(columns=['Isolate'])
    return GS_feat_df

  if combo == 'GYS':
    df_list = [features_df['Isolate'], gene_presc_feat, year_feat, pop_struc_feat, ]
    GYS_feat_df = pd.concat(df_list, axis=1)
    GYS_feat_df = GYS_feat_df.drop(columns=['Isolate'])
    return GYS_feat_df

In [ ]:
# Implementing combo_feat() function created for training data
CTX_GYS_train_df = combo_feat(CTX_Train_test_dic['features_train'],"CTX","GYS")

# looking only at the feature column names for the combination for "GYS" for drug "CTX" for training data
CTX_GYS_train_df.columns

### **5) Creating Random Forest model and training it per feature combination**
In this step we can take a closer look at the structure of a Random Forest. First we will create a function that will train our Random Forest to learn to predict Resistance or Susceptibility for a particular strain. Because our function does not specify how many trees to use, the default is 100 Decision Trees created.

- We have set a random state in our in the Random Forest Classifier in order to get replicable results for this tutorial.

In [ ]:
# creating Random Forest model function
def run_RF(feat_train, lab_train, drug, combo):
  print(drug +" Training combo: "+ combo)
  # creating Random Forest model
  RF = RandomForestClassifier(random_state = 42)
  # Training Random Forest Model
  RF = RF.fit(feat_train, lab_train)
  # Checking number of trees in the model
  print("Number of Decicion Trees in RF model:", len(RF))
  return RF

Below we will implement our function just as in previous parts, but then we will also take a look at one tree of the Random Forest, after training.

In [ ]:
# implementing run_RF() for specific drug feature combination dataframe
RF_CTX_GYS_model = run_RF(CTX_GYS_train_df,CTX_Train_test_dic["labels_train"],"CTX","GYS")
RF_CTX_GYS_model

Now that we have trained our Random Forest model use all features (GYS) to predict for the antibiotic CTX. Below we can graph for example the first tree (index=0). We have 100 trees, and python starts count from 0. Therefore the index would runs from 0 to 99.

In [ ]:
# Setting the index of the tree we are interested in seeing and how many layers to draw
index= 0

# Drawing the tree by setting the Index and the depth of the tree we would like to draw
single_tree = tree.export_graphviz(RF_CTX_GYS_model.estimators_[index], out_file=None,
                                  feature_names=CTX_GYS_train_df.columns,
                                  filled=True, rounded=True,
                                  special_characters=True,
                                  max_depth=None)
graph = graphviz.Source(single_tree)

The code below will store our complete Decision Tree graph as a pdf document. The file will be named based on the index you have chosen to draw, in this case it would be the tree at index 2. Notice that a single tree from our Random Forest can be very big, because the tree will contain several decision thresholds.

In [ ]:
# Saving a single complete decision tree from our Random Forest model into our new directory
graph.render("/content/drive/MyDrive/EColi_ML_Plots/Classification Tree #: "+ str(index))

The code below on the other hand will let us view only the top part of the tree index 0 (only first 3 layers after the root node), you can control the display of the tree by changing the argument **max_depth**.

#### **Parts of a Decision Tree**:

- **Root Node:** displayed at the very top of our Decision tree. It is considered as the most important feature from the random subset of feature columns that our Random Forest (RF) picked. Therefore, each tree in the RF may have a different Root Node.

- **Intermediate Nodes:** the first 2 layers of intermediate nodes are being displayed. Similar to the Root Node it contains decision thresholds that eventually lead to a final prediction.

- **Leaf Nodes:** Not displayed in the trimmed version, but we can observe them in the pdf. The leaf nodes contain the final predictions within one tree of our RF. You can recognize them because there are no decision thresholds.

In [ ]:
# Setting the index of the tree we are interested in seeing and how many layers to draw
index= 0
# Drawing the tree by setting the Index and the depth of the tree we would like to draw
chopped_tree = tree.export_graphviz(RF_CTX_GYS_model.estimators_[index], out_file=None,
                                  feature_names=CTX_GYS_train_df.columns,
                                  filled=False, rounded=True,
                                  special_characters=True,
                                  max_depth=2)
graph = graphviz.Source(chopped_tree)
display(graph)

#### **Information within each Node**

Below is an schematic for each type of node and uses the first Decision Tree of our Random Forest. The type of information displayed in each node, has been labeled (A, B, C and D) only for the root node but it is present in all the root and intermediate nodes, the leaf nodes only contain B, C and D information. Below we will go over what each of the labeled information represent:

![tree-parts.png](https://drive.google.com/uc?export=view&id=1WN05o44tqflVq-GL_2uL_nQuLn3MiNEG)
***
**A) Decision Threshold:** These are essentially Boolean statement about a column feature that represents a threshold. If the statement is TRUE for a particular observation then we follow the left arrow to and if its false FALSE then we follow the right arrow to the next node. The Decision Threshold is only present in the root and intermediate nodes. In the picture example the thresholds would be: <font face="monospace">cutoff_105 $\leq$ 151.0 (root node), cutoff_86 $\leq$ 157.5 (intermediate node), group_3513 $\leq$ 0.5
 (intermediate node)</font>, etc

The features that start with "group" are categorical variables that have already been one-hot encoded, where 1 means that the ortholog gene is present and 0 means that it is not. Therefore if an observation is $\leq$ 0.5 for those thresholds then it means this gene is not present and we would follow the left arrow.

**NOTE:** TRUE is equivalent to **yes (left arrows)** and FALSE equivalent to **no (right arrows)** from the [image](#tree-methods) displayed at the begining of this notebook.

***
**B) Gini Impurity:** is a measure of purity of the classification. It is a number between 0 and 1, where a value of 0 means that all samples belong to one particular class (purest node); whereas a value of 1 means all samples are distributed equally among the classes (least pure node). Here is a link about [Gini Impurity](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity), if you are interested in the calculation and specific formula behind it.

In our graph, we see that the root node has <font face="monospace">Gini = 0.325</font> and as we go lower on the tree the <font face="monospace">Gini = 0.0</font> for the leaf node because the remaining observations in the leaf belong to only one class, the Susceptible (S) one. This means that all the observations that have a <font face="monospace">cutoff_105 $\leq$ 151.0 </font>(root node) and <font face="monospace">cutoff_86 $\leq$ 157.5 </font>(intermediate node) are classified exclusively as Susceptible in this particular tree of our Random Forest model.
***
**C) Samples:** Is the number of unique observations from the original training dataset before the bagging proceedure. In the image we can see that the intermediate node with the threshold cutoff_86 $\leq$ 157.5 has 93 unique samples and it is later split into a leaf node with 28 unique samples and another intermediate node with 65 samples (93 = 28 + 65).
***
**D) Values:** is a list of numbers showing the amount of actual observations classified after the bagging proceedure (random sampling of training observations with replacement + aggregation). We only have 2 classes, so the length of this list is just 2 numbers, we can expect a list of 3 if there are 3 classes to classify, 4 if there are 4 classes, etc.

In the image we can see that as we go down the decision tree the Resistant observations (in red) in the node with the Decision threshold <font face="monospace">group_3513 $\leq$ 0.5</font> has <font color="red">164</font> observations classified  as Resistant, then uppon splitting into 2 intermediate nodes this value is also split into <font color="red">59</font> for the node with threshold <font face="monospace">group_19353 $\leq$ 0.5</font> and <font color="red">105</font>. The same thing happens for the Susceptible split in this node <font color="blue">(938 = 806 + 132) </font>.

**NOTE:** Usually for a regular Decision Tree the sum of the list of numbers in values would add up to the sample number BUT in the case of a Random Forest Decision Tree the **Samples** only count the unique observations, whereas our **Values** include the repeated copies from the bagging proceedure as well .
***
If you are not sure which number corresponds to a particular class, below we can access the index of the values shown in each node. As we can see the first index is Resistant (0) and the second index is Susceptible (1):

In [ ]:
# line to see what classes belong to a particular index in the Values.
RF_CTX_GYS_model.classes_

### **6) Making predictions from Random Forest model**

The code chunks below will attempt to show us how Random Forests make their predictions using in our case 100 Decision Trees. An important distinction between a Decision Tree and Random Forest is to consider how the trees are generated within the Random Forest. In Random Forest there are 2 randomizations happening when they create the trees:
- Random selection of observations from the total training data (rows)
- Random selection of features from the feature combination we have chosen, in this case it would be GYS (columns)

In [ ]:
# creating a function using the model created and trained and the feature combinations from testing data
def predict(RF_combo_Model, features_test):
  labels_pred = RF_combo_Model.predict(features_test)
  return labels_pred

Below we implement first the function combo_feat to specify which testing features we want to use to make our predictions. Then we use our predict function to finally get the predictions made by the Random Forest.

In [ ]:
# Implementing combo_feat() function created for testing data
CTX_GYS_test_df = combo_feat(CTX_Train_test_dic['features_test'],"CTX","GYS")

# Implementation of the predict() function using the feature combination "GY"
CTX_GYS_labels_pred = predict(RF_CTX_GYS_model,CTX_GYS_test_df)

# observe how many predictions were made for each category "R" and "S"
print("Labels predicted: ", np.unique(CTX_GYS_labels_pred, return_counts=True))

The final predictions from the Random Forests were 100 Resistant and 514 Susceptible. However to see how the final decisions were made, we can check how each tree makes predictions for just the first observation in the features test data. The code below provides a counter for trees that voted Susceptible vs the ones that voted Resistant.

In [ ]:
# Code below shows the number of trees that predicted one of the two outcomes
votes_for_R = 0 #we start with 0 trees "voting" for R
votes_for_S = 0 #and 0 trees "voting" for S

#loop over each of the trees in the random forest
for i in range(0,len(RF_CTX_GYS_model)):
  #get a prediction from one tree
  pred = RF_CTX_GYS_model.estimators_[i].predict(CTX_GYS_test_df.loc[0,:].to_numpy().reshape(1,-1))

  # if the prediction is 0 then it is "R"
  if pred == 0.00:
    votes_for_R = votes_for_R + 1

  #if the prediction is 1 then it is "S"
  else:
    votes_for_S = votes_for_S + 1
# Counting the number of trees that predicted R and the ones that predicted S
print("Number of trees that voted Resistance: ",votes_for_R)
print("Number of trees that voted Susceptible: ",votes_for_S)

Below we can check the prediction by the Random Forest, for the first observation as well. As expected, because the majority of the trees voted for Susceptible (S), we can see that our Random Forest ends up predicting S as well.

In [ ]:
# prediction using the second value of our test features
pred = RF_CTX_GYS_model.predict(CTX_GYS_test_df.loc[0,:].to_numpy().reshape(1,-1))
print("Prediction by Random Forest: ", pred)

### **7) Evaluating our model using a confusion matrix and metrics**

**a) Creating evaluate() function and implementing it:**

Similarly to the previous notebook, we will evaluate our RF model by using a Confusion Matrix and respective metrics. Below is a quick review of these, remember that there is one Accuracy score, but Recall and Presicion should have as many sets as classes our model its trained to predict:

|<font size=3>Metrics|<font size=3>General formula|<font size=3> Formula for 2 classes|
|--|:-:|:-:
|<font size=3>**Accuracy**|<font size=3>$\frac{Correctly \ classified}{All \ Predicted}$|<font size=3>$\frac{TP + TN}{TP + TN + FN + FP}$|
|<font size=3>**R Recall:**|<font size=3>$\frac{Correctly \ classified \ as \ R}{All \ Actual \ R}$|<font size=3>$\frac{TP}{TP + FN}$|
|<font size=3>**R Precision:**|<font size=3>$\frac{Correctly \ classified \ as \ R}{All \ Predicted \ R}$|<font size=3>$\frac{TP}{TP + FP}$|

In [ ]:
# Creating a function that evaluates our model using our actual and predicted data
def evaluate(RF_combo_model, labels_test, labels_pred, cf= True, show_results=True):
  report = classification_report(labels_test, labels_pred, output_dict = True)
  if cf == True:
    cm = confusion_matrix(labels_test, labels_pred, labels=RF_combo_model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=RF_combo_model.classes_)
    disp.plot()
    plt.show()
  if show_results == True:
    print("Results")
    print('Accuracy:',report['accuracy'])
    print('R recall:',report['R']['recall'])
    print('S recall:',report['S']['recall'])
    print('R precision:',report['R']['precision'])
    print('S precision:',report['S']['precision'])
  return [report['accuracy'], report['R']['recall'], report['S']['recall'], report['R']['precision'], report['S']['precision']]

In [ ]:
# implementing the evaluate() function
Model_Report = evaluate(RF_CTX_GYS_model, CTX_Train_test_dic['labels_test'],CTX_GYS_labels_pred)

**b) Create a feature importance graph showing which features were the most important**

The gini impurity is a metric used to determine which features are more important than others. Below we will learn to graph the feature importance for the model we have been working on.

In [ ]:
# getting feature importance for our Random Forest model trained using GYS feature combo for CTX drug
importance = RF_CTX_GYS_model.feature_importances_

# getting the 20 highest feature importance out of 18291 TOTAL features
indices = np.argsort(importance)[-20:]
highest_feat_importance = importance[indices]

# create a barchart of the 20 most important features
sns.set_theme()
plt.barh([x for x in range(len(highest_feat_importance))], highest_feat_importance, tick_label=CTX_GYS_train_df.columns[indices])
plt.title("20 most important features for CTX_GYS_model")

# saving feature importance bargraph and display in notebook
plt.savefig('/content/drive/My Drive/EColi_ML_Plots/RandomForest_CTX_GYS_feat_importance.jpg',dpi=400, bbox_inches="tight")
plt.show()

Based on the feature importance bar chart it seems that considering all GYS features, the column feature **wbuC** ,which belongs to the Gene Presence or Absence Dataset (**G**), seems to have the highest importance in this particular model for the antibiotic CTX. This probably means that strains with this gene are more likely to be resistant to the drug Cefotaxime (CTX).

### **8) Use all functions and evaluate every drug in every feature combination!**

**NOTE:** Code below is the same as in previous notebook

#### **a) Lets recall the list of drugs we have available and the combination of features we are interested in**

In [ ]:
# let's check all drugs
drug_list = All_Drugs_df.iloc[:,1:13].columns
print(drug_list)

# let's see all combinations we are interested in
print(combo_list)

#### **b) Create a loop that will go through all our functions using the lists above**

In [ ]:
# Lets use all our functions this time and save our report into a single data structure
RF_model_metrics = {}

for drug in drug_list:
  print(drug)
  # splits each drug df into a dictionary with testing and training data
  Test_Train_dic = Split_train_test (drug)
  for combo in combo_list:
    # Training each drug_combo features
    labels_train = Test_Train_dic["labels_train"]
    # create corresponding feature_df for training
    features_train = combo_feat(Test_Train_dic["features_train"], drug, combo)
    # runs Random Forest model using the corresponding training feature_df
    RF_combo_model = run_RF(features_train, labels_train, drug, combo)

    # Predicting each drug_combo features
    # create corresponding feature_df for testing
    features_test = combo_feat(Test_Train_dic["features_test"], drug, combo)
    # generate predictions based on the feature combination tested
    labels_pred = predict(RF_combo_model, features_test)

    # Evaluating our models
    labels_test = Test_Train_dic["labels_test"]
    report = evaluate(RF_combo_model, labels_test, labels_pred, cf= False, show_results=False)
    RF_model_metrics[drug+"_"+combo] = report

    print(report)

#### **c) Store the metrics report for all drugs and features combinations as a csv file**

In [ ]:
# convert dictionary into a dataframe
RF_metrics = pd.DataFrame.from_dict(RF_model_metrics, orient='index', columns=["Accuracy", "R_recall", "S_recall", "R_precision", "S_precision"]).reset_index()
RF_metrics = RF_metrics.rename(columns = {'index':'Drug_combo'})

# saving our metric results into a CSV file
RF_metrics.to_csv(filepath+"RF_metrics_df.csv", index= False)
RF_metrics


Great! We have created and saved all the Random Forest results for every antibiotic and combination of features we were interested in. Now that we have a basic understanding of tree methods in general. It is time to learn about another tree method: [Extreme Gradient Boosted Tree](https://colab.research.google.com/drive/1Ty-l-xSu3t-qbxfx2kN8jMSIOLxC9cYd?usp=sharing).